In [86]:
import sys
import os

# Assume your modules are in the 'src' directory under the current notebook directory
# Get the absolute path of the parent directory
parent_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))

# Add it to sys.path if not already included
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)

from utils import common
from utils import extract
from utils import handlebars

# necessary sometimes when importing in VSCode
import importlib
importlib.reload(common)
importlib.reload(extract)
importlib.reload(handlebars)

<module 'utils.handlebars' from '/Users/nicholasbern/src/holocron/utils/handlebars.py'>

In [87]:
import configparser
from openai import OpenAI


config = configparser.ConfigParser()
config.read('../config.ini')

client = OpenAI(
    api_key=config['OPENAI']['OPENAI_API_KEY']
)

In [89]:
import itertools


memory_temperature = 0.25
chat_temperature = 0.25

memory_model = "gpt-4o"
chat_model = "gpt-4o"

def get_memories(prompt_data, questions, prompt_template):
    memories = []

    for question in questions["speaking_style"]:
        question_message = {
            "role": "user",
            "first_name": "Lore",
            "full_name": "Lore Master",
            "message": question
        }
        prompt_data_for_memories = prompt_data.copy()
        prompt_data_for_memories["messages"] = [question_message]

        memory_result = client.chat.completions.create(
            messages=handlebars.make_prompt(prompt_template, prompt_data_for_memories),
            model=memory_model,
            temperature=memory_temperature
        )

        memories += question_message
        memories += {
            "role": "assistant",
            "first_name": "", # not used
            "full_name": "", # not used
            "message": memory_result.choices[0].content
        }
    
    return memories

conversation_filepath = "../data/multi-user-test-conversations.jsonl"
conversations = common.read_jsonl_file(conversation_filepath)

questions_filepath = "../data/questions.json"
questions = common.read_json_file(questions_filepath)

prompt_template_filepath = "../data/prompt-template.hbs"
prompt_template = common.read_file(prompt_template_filepath)

how_many_conversations = 1

results = []

for conversation in itertools.islice(conversations, how_many_conversations):
    prompt_data = extract.extract_prompt_data(conversation)
    messages = prompt_data["messages"]

    memories = get_memories(prompt_data, questions, prompt_template)

    no_memories_result = client.chat.completions.create(
        messages=handlebars.make_prompt(prompt_template, prompt_data),
        model=chat_model,
        temperature=chat_temperature
    ).choices[0].message["content"]
    with_memories_result = client.chat.completions.create(
        messages=handlebars.make_prompt(prompt_template, {
            **prompt_data,
            "memories": memories + prompt_data["messages"]
        }),
        model=chat_model,
        temperature=chat_temperature
    ).choices[0].message["content"]

    results.append({
        "conversation": conversation,
        "no_memory": no_memories_result,
        "with_memory": with_memories_result
    })

common.save_json_file("../data/memories-test-results.json", results)
common.save_jsonl_file("../data/conversations.json", results)